In [1]:
%autosave 0 

Autosave disabled


In [115]:
# standard imports
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
import os

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from wrangle import missing_by_row, get_fences

from env import get_connection


In [110]:
def get_titanic():
    
    filename = 'zillow.csv'
    
    if os.path.isfile(filename):
        
        return pd.read_csv(filename)
    
    else:
        
        url = get_connection('zillow')
        
        query = '''
                -- here, I want to ensure that I am selecting
                -- properties that have a transaction in 2017,
                -- the most recent version of those properties
                -- from there, I want to get the logerror for the zestimate
                -- and any potential supplementary information 
                -- available in the other tables
                -- SELECT: everything from properties aliased as prop
                SELECT prop.*,
                -- predictions_2017 : logerror and transactiondate
                predictions_2017.logerror,
                predictions_2017.transactiondate,
                -- all the other supplementary stuff
                air.airconditioningdesc,
                arch.architecturalstyledesc,
                build.buildingclassdesc,
                heat.heatingorsystemdesc,
                land.propertylandusedesc,
                story.storydesc,
                type.typeconstructiondesc
                FROM properties_2017 prop
                JOIN (
                    SELECT parcelid, MAX(transactiondate) AS max_transactiondate
                    FROM predictions_2017
                    GROUP BY parcelid
                    ) pred USING(parcelid)
                JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
                                  AND pred.max_transactiondate = predictions_2017.transactiondate
                LEFT JOIN airconditioningtype air USING(airconditioningtypeid)
                LEFT JOIN architecturalstyletype arch USING(architecturalstyletypeid)
                LEFT JOIN buildingclasstype build USING(buildingclasstypeid)
                LEFT JOIN heatingorsystemtype heat USING(heatingorsystemtypeid)
                LEFT JOIN propertylandusetype land USING(propertylandusetypeid)
                LEFT JOIN storytype story USING(storytypeid)
                LEFT JOIN typeconstructiontype type USING(typeconstructiontypeid)
                WHERE propertylandusedesc = "Single Family Residential"
                    AND transactiondate <= '2017-12-31'
                    AND prop.longitude IS NOT NULL
                    AND prop.latitude IS NOT NULL
                '''
        
        df = pd.read_sql(query, url)
        
        df.to_csv(filename, index = 0)
        
        return df

In [111]:
df = get_titanic()
df

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.50,4.00,NaN,NaN,3.50,...,"60,590,630,072,012.00",0.03,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.00,2.00,NaN,NaN,1.00,...,"61,110,010,023,006.00",0.06,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.00,3.00,NaN,NaN,2.00,...,"60,590,218,022,012.00",0.01,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.00,4.00,NaN,8.00,3.00,...,"60,373,001,001,006.00",-0.10,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,781532,12095076,1.00,NaN,NaN,3.00,4.00,NaN,9.00,3.00,...,"60,374,608,001,014.00",-0.00,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52314,2274245,12412492,NaN,NaN,NaN,2.00,4.00,NaN,6.00,2.00,...,"60,375,548,022,001.00",0.00,2017-09-19,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
52315,673515,11000655,NaN,NaN,NaN,2.00,2.00,NaN,6.00,2.00,...,"60,371,014,003,002.00",0.02,2017-09-20,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
52316,2968375,17239384,NaN,NaN,NaN,2.00,4.00,NaN,NaN,2.00,...,"61,110,084,022,016.00",0.01,2017-09-21,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
52317,1843709,12773139,1.00,NaN,NaN,1.00,3.00,NaN,4.00,1.00,...,"60,374,338,022,005.00",0.04,2017-09-21,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [112]:
missing_by_col(df)


id                                  0
parcelid                            0
airconditioningtypeid           38704
architecturalstyletypeid        52249
basementsqft                    52272
bathroomcnt                         0
bedroomcnt                          0
buildingclasstypeid             52319
buildingqualitytypeid           18665
calculatedbathnbr                 135
decktypeid                      51930
finishedfloor1squarefeet        47948
calculatedfinishedsquarefeet       81
finishedsquarefeet12              246
finishedsquarefeet13            52319
finishedsquarefeet15            52319
finishedsquarefeet50            47948
finishedsquarefeet6             52154
fips                                0
fireplacecnt                    45086
fullbathcnt                       135
garagecarcnt                    34335
garagetotalsqft                 34335
hashottuborspa                  50805
heatingorsystemtypeid           18470
latitude                            0
longitude   

In [114]:
result_df = missing_by_row(df)
print(result_df)

    num_cols_missing  percent_cols_missing  num_rows
0                 23                 34.00         2
1                 24                 35.00        12
2                 25                 37.00        11
3                 26                 38.00        30
4                 27                 40.00       177
5                 28                 41.00       390
6                 29                 43.00      2527
7                 30                 44.00      2199
8                 31                 46.00      5989
9                 32                 47.00      8885
10                33                 49.00     11967
11                34                 50.00     11157
12                35                 51.00      3469
13                36                 53.00      4138
14                37                 54.00      1020
15                38                 56.00       223
16                39                 57.00        29
17                40                 59.00    

In [116]:
# Assuming you have a DataFrame named 'my_dataframe'
# Call the get_fences function for a specific column 'my_column' with the default value of k
lower_bound, upper_bound = get_fences(df, 'my_column')

# Print the lower and upper bounds
print(f"Lower Bound: {lower_bound}")
print(f"Upper Bound: {upper_bound}")

KeyError: 'my_column'